<a href="https://colab.research.google.com/github/MarkovMarkowitz/MarkovMarkowitz/blob/main/Blueshift_Scalping_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
"""
    Title: ATR scalping strategy
    Description: This strategy uses ATR indicator for the short term volatility change
                 which helps to take long or short positions depending on
                 where the price is moving
    Style tags: Momentum
    Asset class: Equities
    Dataset: US Equities
    Start and end date: Keep the backtesting date range small (1 or 2 years)
                        to avoid time out error
"""



from blueshift.api import(symbol,
                        order_target_percent,
                        order,
                        schedule_function,
                        date_rules,
                        time_rules,
                        get_datetime
                        )

import numpy as np
import talib
from datetime import datetime


def initialize(context):

    # Define symbol
    context.security = symbol('PXD')

    # Define stop-loss and take-profit thesholds
    context.stop_loss_threshold = 0.01
    context.take_profit_threshold = 0.02

    context.stop_loss = np.nan
    context.take_profit = np.nan

    context.position = 0

    context.eod_flag = False

    # Schedule strategy logic
    schedule_function(square_off,
                     date_rule=date_rules.every_day(),
                     time_rule=time_rules.market_close(minutes=10))


def before_trading_start(context, data):
    context.eod_flag = False


def handle_data(context, data):

    if not context.eod_flag:

        # Fetch 1 min data for the above security
        try:
            prices = data.history(
                context.security, ['open', 'high', 'low', 'close','volume'], 60, '1m')
            prices = prices[prices.volume!=0]
            if len(prices) < 60:
                return
        except IndexError:
            return

        # Calculate the Average True Range(ATR)
        prices['ATR'] = talib.ATR(prices['high'].values, prices['low'].values,
                                  prices['close'].values, timeperiod=30)

        # Calculate the rolling mean of ATR
        prices['ATR_MA_5'] = prices['ATR'].rolling(5).mean()

        # Flag the minutes where ATR breaks out its rolling mean
        ATR_breakout = prices['ATR'][-1] > prices['ATR_MA_5'][-1]

        # Check if the fourth candle is higher than the highest of the previous 3 candle
        four_candle_high = prices.iloc[-1, 1] >= prices.iloc[-4:, 1].max()

        # Check if the fourth candle is lower than the lowest of the previous 3 candles
        four_candle_low = prices.iloc[-1, 2] <= prices.iloc[-4:, 2].min()

        # Long entry and exit condition
        long_entry = ATR_breakout and four_candle_high
        long_exit = prices['close'][-1] < context.stop_loss or prices['close'][-1] > context.take_profit


        # Short entry and exit condition
        short_entry = ATR_breakout and four_candle_low
        short_exit = prices['close'][-1] > context.stop_loss or prices['close'][-1] < context.take_profit


        # Place the order
        if long_entry and context.position == 0:
            print("{} Opening long position in: {}".format(get_datetime(), context.security.symbol))
            order_target_percent(context.security, 1)
            context.position = 1
            context.stop_loss = prices['close'][-1] * \
                (1-context.stop_loss_threshold)
            context.take_profit = prices['close'][-1] * \
                (1+context.take_profit_threshold)

        elif short_entry and context.position == 0:
            print("{} Opening short position in: {}".format(get_datetime(), context.security.symbol))
            order_target_percent(context.security, -1)
            context.position = -1
            context.stop_loss = prices['close'][-1] * \
                (1+context.stop_loss_threshold)
            context.take_profit = prices['close'][-1] * \
                (1-context.take_profit_threshold)

        elif long_exit and context.position == 1:
            print("{} Closing long position in: {}".format(get_datetime(), context.security.symbol))
            order_target_percent(context.security, 0)
            context.position = 0
            context.stop_loss = np.nan
            context.take_profit = np.nan

        elif short_exit and context.position == -1:
            print("{} Closing short position in: {}".format(get_datetime(), context.security.symbol))
            order_target_percent(context.security, 0)
            context.position = 0
            context.stop_loss = np.nan
            context.take_profit = np.nan


def square_off(context, data):

    # Exit position
    if not context.position == 0:
        order_target_percent(context.security, 0)
        context.position = 0
        context.stop_loss = np.nan
        context.take_profit = np.nan
    context.eod_flag = True

ERROR: Could not find a version that satisfies the requirement blueshift.api (from versions: none)
ERROR: No matching distribution found for blueshift.api


ModuleNotFoundError: ignored